# Project 3
## Hanan Sukenik

### Part 1- Instrumenting the API server to log events to Kafka

We will begin our work by using a Docker Compose .yml file to configure and start multiple Docker containers needed for our project. In this case, we used a Docker Compose file containing Zookeeper, Kafka, Cloudera (HDFS), Spark, Hive, Presto and the midsw250 base container.
We will examine the Docker-Compose file, spin up the cluster and examine the kafka logs. Finally, we will make sure all of the containers are up.

```bash
cd ~/w205/project-3-hanansuk
cp ../course-content/13a/docker-compose.yml .
vim docker-compose.yml 
docker-compose up -d
docker-compose logs -f kafka
docker-compose ps
```

We will initiate flask:

```bash
docker-compose exec mids env FLASK_APP=/w205/project-3-hanansuk/game_api.py flask run --host 0.0.0.0
```

Next, we will create a new topic on Kafka, that represents the events we are about to send to the topic. In this case, the topic is "events", as the messages we represent different events. Next, we will use kafkacat to set up Kafka to watch for streaming events.
Finally, we will (again) make sure the containers are still running.
* Note: We will be using bootstrap-server as Zookeeper seems to no longer be supported

```bash
docker-compose exec kafka kafka-topics --create --topic events -partitions 1 --replication-factor 1 --if-not-exists --bootstrap-server kafka:29092
docker-compose exec mids kafkacat -C -b kafka:29092 -t events -o beginning
docker-compose ps
``` 

### Part 2- Creating events and extracting them from Kafka To HDFS, while registering the tables with Hive

We will be using 2 different Spark scripts to extract data from Kafka to HDFS and register tables for them with Hive. The two scripts will represent the two different events- Purchase Sword and Join Guild.
We will be using two seperate terminals for each of the scripts.

```bash
docker-compose exec spark spark-submit /w205/project-3-hanansuk/stream_and_hive_sword.py
docker-compose exec spark spark-submit /w205/project-3-hanansuk/stream_and_hive_guild.py
```

While both of those are running, we will initiate events using Apache Bench. In order to use that, we will use the gameSim.sh bash script to emulate Purchase Sword and Join Guild events.
We will run this script twice for more events.

```bash
bash gameSim.sh
```

Now, let's make sure the events were written to Hadoop. We will first make sure folders were created for both events, and then make sure that each folder actually contains events.

```bash
docker-compose exec cloudera hadoop fs -ls /tmp
docker-compose exec cloudera hadoop fs -ls /tmp/sword_purchases
docker-compose exec cloudera hadoop fs -ls /tmp/join_guilds
```

Output:
```bash
Found 7 items
drwxrwxrwt   - root   supergroup          0 2021-12-05 23:51 /tmp/checkpoints_for_join_guilds
drwxrwxrwt   - root   supergroup          0 2021-12-05 23:51 /tmp/checkpoints_for_sword_purchases
drwxrwxrwt   - mapred mapred              0 2016-04-06 02:26 /tmp/hadoop-yarn
drwx-wx-wx   - hive   supergroup          0 2021-12-05 23:51 /tmp/hive
drwxrwxrwt   - root   supergroup          0 2021-12-05 23:52 /tmp/join_guilds
drwxrwxrwt   - mapred hadoop              0 2016-04-06 02:28 /tmp/logs
drwxrwxrwt   - root   supergroup          0 2021-12-05 23:52 /tmp/sword_purchases

Found 6 items
drwxr-xr-x   - root supergroup          0 2021-12-05 23:52 /tmp/sword_purchases/_spark_metadata
-rw-r--r--   1 root supergroup        688 2021-12-05 23:52 /tmp/sword_purchases/part-00000-17e6e0ab-e8c2-49cd-abed-b4fb6fb17275-c000.snappy.parquet
-rw-r--r--   1 root supergroup       2909 2021-12-05 23:52 /tmp/sword_purchases/part-00000-2d970bf3-f2eb-4d00-9893-6552c380fb49-c000.snappy.parquet
-rw-r--r--   1 root supergroup       2927 2021-12-05 23:52 /tmp/sword_purchases/part-00000-4c0aae91-566c-4fbb-9087-984d3aa1c7f6-c000.snappy.parquet
-rw-r--r--   1 root supergroup        688 2021-12-05 23:52 /tmp/sword_purchases/part-00000-747659d2-29ec-466b-8a93-057ec24e1008-c000.snappy.parquet
-rw-r--r--   1 root supergroup       2449 2021-12-05 23:52 /tmp/sword_purchases/part-00000-ad0d843d-2eb2-4b43-960d-526fa4f9a3c5-c000.snappy.parquet

Found 6 items
drwxr-xr-x   - root supergroup          0 2021-12-05 23:52 /tmp/join_guilds/_spark_metadata
-rw-r--r--   1 root supergroup       2488 2021-12-05 23:52 /tmp/join_guilds/part-00000-263bdd2e-a73a-47b5-9d37-15864ebaf6db-c000.snappy.parquet
-rw-r--r--   1 root supergroup        688 2021-12-05 23:52 /tmp/join_guilds/part-00000-332c7061-d355-4ebd-8ac2-d295be4bd55f-c000.snappy.parquet
-rw-r--r--   1 root supergroup       2755 2021-12-05 23:52 /tmp/join_guilds/part-00000-5ecd244d-68dc-47ca-a7d4-6917208c3f62-c000.snappy.parquet
-rw-r--r--   1 root supergroup       2537 2021-12-05 23:52 /tmp/join_guilds/part-00000-62a0d3a0-f8af-496b-8a74-ee53788147d4-c000.snappy.parquet
-rw-r--r--   1 root supergroup        688 2021-12-05 23:52 /tmp/join_guilds/part-00000-64daaf36-046b-4f79-bb77-af654773fae5-c000.snappy.parquet
```

### Part 3- Reading the Data with Presto and Querying

We will run Presto to start looking into the data. We'll start by loading Presto.

```bash
docker-compose exec presto presto --server presto:8080 --catalog hive --schema default
```

#### Next, we will run some queries to examine the data:



##### Query 1
* Looking at the tables on Presto

Query:
```sql
show tables;
```

Output:
```sql
 join_guilds     
 sword_purchases 
(2 rows)
```


##### Query 2
* Describing the sword_purchases table

Query:
```sql
describe sword_purchases;
```

Output:
```sql
   Column   |  Type   | Comment 
------------+---------+---------
 raw_event  | varchar |         
 timestamp  | varchar |         
 accept     | varchar |         
 host       | varchar |         
 user-agent | varchar |         
 event_type | varchar |         
(6 rows)
```


##### Query 3
* Describing the join_guilds table

Query:
```sql
describe join_guilds;
```

Output:
```sql
   Column   |  Type   | Comment 
------------+---------+---------
 raw_event  | varchar |         
 timestamp  | varchar |         
 accept     | varchar |         
 host       | varchar |         
 user-agent | varchar |         
 event_type | varchar |         
(6 rows)
```

* Counting the number of sword_purchase events

Query:
```sql
select count(*) from sword_purchases;
```

Output:
```sql
 _col0 
-------
   160 
(1 row)
```



##### Query 4
* Counting the number of join_guild events

Query:
```sql
select count(*) from join_guilds;
```

Output:
```sql
 _col0 
-------
   130 
(1 row)
```



##### Query 5
* Looking into the first sword_purchase events

Query:
```sql
select * from sword_purchases limit 5;
```

Output:
```sql
                                                     raw_event                                                     |        timestamp        | accept |        host         |   user-agent    |   event_typ
-------------------------------------------------------------------------------------------------------------------+-------------------------+--------+---------------------+-----------------+------------
 {"Host": "user101.comcast.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"} | 2021-12-05 23:52:08.548 | */*    | user101.comcast.com | ApacheBench/2.3 | purchase_sw
 {"Host": "user101.comcast.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"} | 2021-12-05 23:52:08.582 | */*    | user101.comcast.com | ApacheBench/2.3 | purchase_sw
 {"Host": "user101.comcast.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"} | 2021-12-05 23:52:08.592 | */*    | user101.comcast.com | ApacheBench/2.3 | purchase_sw
 {"Host": "user101.comcast.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"} | 2021-12-05 23:52:08.595 | */*    | user101.comcast.com | ApacheBench/2.3 | purchase_sw
 {"Host": "user101.comcast.com", "event_type": "purchase_sword", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"} | 2021-12-05 23:52:08.601 | */*    | user101.comcast.com | ApacheBench/2.3 | purchase_sw
(5 rows)
```



##### Query 6
* Looking into the first join_guilds events

Query:
```sql
select * from join_guilds limit 5;
```

Output:
```sql
                                                   raw_event                                                   |        timestamp        | accept |        host         |   user-agent    | event_type 
---------------------------------------------------------------------------------------------------------------+-------------------------+--------+---------------------+-----------------+------------
 {"Host": "user101.comcast.com", "event_type": "join_guild", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"} | 2021-12-05 23:52:17.788 | */*    | user101.comcast.com | ApacheBench/2.3 | join_guild 
 {"Host": "user101.comcast.com", "event_type": "join_guild", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"} | 2021-12-05 23:52:17.797 | */*    | user101.comcast.com | ApacheBench/2.3 | join_guild 
 {"Host": "user101.comcast.com", "event_type": "join_guild", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"} | 2021-12-05 23:52:17.801 | */*    | user101.comcast.com | ApacheBench/2.3 | join_guild 
 {"Host": "user101.comcast.com", "event_type": "join_guild", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"} | 2021-12-05 23:52:17.808 | */*    | user101.comcast.com | ApacheBench/2.3 | join_guild 
 {"Host": "user101.comcast.com", "event_type": "join_guild", "Accept": "*/*", "User-Agent": "ApacheBench/2.3"} | 2021-12-05 23:52:17.813 | */*    | user101.comcast.com | ApacheBench/2.3 | join_guild 
(5 rows)
```



##### Query 7
* Counting the number of join_guild events where the host is 'mossad.surveillance.gov'

Query:
```sql
select count(*) from join_guilds where host = 'mossad.surveillance.gov';
```

Output:
```sql
 _col0 
-------
    40 
(1 row)
```



##### Query 8
* Counting the number of sword_purchase events where the host is 'mossad.surveillance.gov'

Query:
```sql
select count(*) from sword_purchases where host = 'mossad.surveillance.gov';
```

Output:
 _col0 
-------
    20 
(1 row)
```

### Final Steps

The data is ready to be explored further by a data scientist working for the game developement company.

To wrap things up, we will take our Docker-Compose down:
```bash
docker-compose down
```